In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
from sklearn.impute import SimpleImputer

# Load the dataset
file_path = 'C:/Users/marya/OneDrive/Desktop/Classification_project1/preprocessed_green_tripdata_2015-07.csv'
data = pd.read_csv(file_path)

# Create the price_category column based on total_amount quantiles
quantiles = data['total_amount'].quantile([0.33, 0.66])
def categorize_price(amount):
    if amount <= quantiles[0.33]:
        return 'Low'
    elif amount <= quantiles[0.66]:
        return 'Medium'
    else:
        return 'High'

data['price_category'] = data['total_amount'].apply(categorize_price)

# Separate the target variable before encoding
y = data['price_category']

# Drop or convert non-numeric columns
non_numeric_columns = ['lpep_pickup_datetime', 'lpep_dropoff_datetime', 'store_and_fwd_flag', 'pickup_datetime', 'dropoff_datetime']
data = data.drop(columns=non_numeric_columns)

# Convert categorical variables to numeric using one-hot encoding
X = pd.get_dummies(data.drop('price_category', axis=1), drop_first=True)

# Check for missing values and impute them
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [20]:
# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
rf_classifier.fit(X_train, y_train)

# Make predictions
y_pred = rf_classifier.predict(X_test)

In [21]:
# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted')

# Generate classification report
class_report = classification_report(y_test, y_pred, output_dict=True)
report_df = pd.DataFrame(class_report).transpose()

# Generate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Display the results
print("Class  Accuracy  Precision    Recall")
for i, category in enumerate(['Low', 'Medium', 'High']):
    print(f"{i}     {category}  {report_df.loc[category, 'precision']:.6f}  {report_df.loc[category, 'recall']:.6f}")

print(f"Testing Accuracy: {accuracy}")
print(f"Testing Precision: {precision}")
print(f"Testing Recall: {recall}")
print("Confusion Matrix:")
print(conf_matrix)

# Detailed confusion matrix breakdown
print(f"True Low:  {conf_matrix[0, 0]} False Medium:  {conf_matrix[0, 1]} False High:  {conf_matrix[0, 2]}")
print(f"False Low:  {conf_matrix[1, 0]} True Medium:  {conf_matrix[1, 1]} False High:  {conf_matrix[1, 2]}")
print(f"False Low:  {conf_matrix[2, 0]} False Medium:  {conf_matrix[2, 1]} True High:  {conf_matrix[2, 2]}")

Class  Accuracy  Precision    Recall
0     Low  0.999954  1.000000
1     Medium  0.999898  0.999949
2     High  1.000000  0.999902
Testing Accuracy: 0.9999513334631107
Testing Precision: 0.9999513358697651
Testing Recall: 0.9999513334631107
Confusion Matrix:
[[20395     0     2]
 [    0 21688     0]
 [    0     1 19558]]
True Low:  20395 False Medium:  0 False High:  2
False Low:  0 True Medium:  21688 False High:  0
False Low:  0 False Medium:  1 True High:  19558
